In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from typing import Literal
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_core.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write an email to the given recipient with the given subject and content."""
    return f"Email sent to {to} with subject {subject} and content: {content}"

@tool
def schedule_meeting(attendees: list[str], subject: str, duration_minutes: int, preferred_day: datetime, start_time: int) -> str:
    """Schedule a meeting"""
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled for {date_str} from {start_time}:00 to {start_time + duration_minutes // 60}:{duration_minutes % 60} with {attendees}"


@tool
def check_calendar_availability(day: str) -> str:
    """Check the availability of the calendar on the given day"""
    return f"Available slots on {day}: 9:00AM, 2:00PM, 4:00PM"

@tool
class Done(BaseModel):
    done: bool

In [6]:
# Triaging router
from langgraph.graph import MessagesState


class State(MessagesState):
    email_input: dict
    classification_decision: Literal["ignore", "respond", "notify"]

In [1]:
from pydantic import BaseModel, Field
from src.personal_agent.prompts import triage_system_prompt, triage_user_prompt, default_triage_instructions, default_background
from src.personal_agent.utils import parse_email, format_email_markdown
from langchain.chat_models import init_chat_model
from langgraph.graph import END
from langgraph.types import Command


In [2]:
from rich.markdown import Markdown
Markdown(triage_system_prompt)

< Role > Your role is to triage incoming emails based upon instructs and background information below. </ Role >   

< Background > {background}. </ Background >                                                                       

< Instructions > Categorize each email into one of three categories:                                               

 1 IGNORE - Emails that are not worth responding to or tracking                                                    
 2 NOTIFY - Important information that worth notification but doesn't require a response                           
 3 RESPOND - Emails that need a direct response Classify the below email into one of these categories. </          
   Instructions >                                                                                                  

< Rules > {triage_instructions} </ Rules >

In [ ]:
class RouterSchema(BaseModel):
    """Schema for the router"""
    reasoning: str = Field(description="Step by step reasoning for the classification decision")

    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, 'respond' for emails that need a reply and 'notify' for important information that doesn't need a response")

llm = init_chat_model(model="gpt-4o-mini", temperature=0)
llm_router = llm.with_structured_output(RouterSchema)

def triage_router(state: State) -> Command[Literal["response_agent", "__end__"]]:
    """Analyze the email and decide whether to ignore, respond or notify"""
    author, to, subject, email_thread = parse_email(state["email_input"])
    system_prompt = triage_system_prompt.format(
        background=default_background,
        triage_instructions=default_triage_instructions
    )
    user_prompt = triage_user_prompt.format(
        email_input=format_email_markdown(subject, author, to, email_thread)
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = llm_router.invoke(messages)

    if response.classification == "ignore":
        print("Classification: IGNORE. This email will be ignored.")
        goto = END
        update = {'classification_decision': 'ignore'}
    elif response.classification == "respond":
        print("Classification: RESPOND. This email will be replied to.")
        goto = "response_agent"
        update = {
            messages: [
                "role": "user",
                "content": f"Respond to the following email: {format_email_markdown(subject, author, to, email_thread)}"
            ],
            classification_decision: "respond"
        }
    elif response.classification == "notify":
        print("Classification: NOTIFY. This email will be notified.")
        goto = END
        update = {
            classification_decision: "notify"
        }
    else:
        raise ValueError("Invalid classification")
    return Command(goto=goto, update=update)

SyntaxError: invalid syntax (2714235268.py, line 37)